In [15]:
import sqlite3

In [16]:
conn = sqlite3.connect('database.sqlite')

In [17]:
cursor= conn.cursor()

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print('Tables:', [t[0] for t in cursor.fetchall()])

# Question 1. Write a SQL query that lists all the players born between 1987 and 1990 inclusive, sort them from the oldest to the youngest.  : 

In [ ]:
print(' Players Name, Birthday')
print('-'*40)
qry1 = f'SELECT ROW_NUMBER() OVER (ORDER BY date(birthday)), player_name, birthday \
         FROM Player \
         WHERE birthday between  {1987} and {1990}\
         '
for ix, name, bday in cursor.execute(qry1):
    print('{},{}, {}'.format(ix, name, bday))

# Question 2: Write a SQL query that ranks all countries and leagues based on the total amount of total goals scored per game in the whole dataset. Sort them by the largest to the smallest amount of goals. 

In [ ]:
print(' Country, League Name, Total Goals Scored')
print('-'*50)
qry2 = f'\
    SELECT Country.name, League.Name, SUM(Match.home_team_goal + Match.away_team_goal)\
        AS Goals FROM \
        Match INNER JOIN League \
        ON Match.league_id = League.id \
        INNER JOIN Country ON League.country_id = Country.id  \
        GROUP BY Country.name, League.Name\
        ORDER BY Goals DESC'
for res in cursor.execute(qry2):
    print(res)

# Question 3: Write a SQL query that ranks all teams by the average of all their attributes (not the players' attributes), sort them from best to worst.  

In [ ]:
print(' Team Long Names, Average of Attributes')
print('-'*50)
qry3 = 'SELECT t.team_long_name, \
       (buildupplayspeed+buildUpPlayDribbling+buildUpPlayPassing+chanceCreationPassing+chanceCreationCrossing+chanceCreationShooting+defencePressure+defenceAggression+defenceTeamWidth)/9  as average \
FROM   team t \
       JOIN (SELECT team_fifa_api_id, \
                    team_api_id, \
                    Avg(buildupplayspeed) AS buildUpPlaySpeed ,\
                    Avg(buildUpPlayDribbling) AS buildUpPlayDribbling , \
                    Avg(buildUpPlayPassing) AS buildUpPlayPassing , \
                    Avg(chanceCreationPassing) AS chanceCreationPassing , \
                    Avg(chanceCreationCrossing) AS chanceCreationCrossing , \
                    Avg(chanceCreationShooting) AS chanceCreationShooting , \
                    Avg(defencePressure) AS defencePressure , \
                    Avg(defenceAggression) AS defenceAggression , \
                    Avg(defenceTeamWidth) AS defenceTeamWidth \
             FROM   team_attributes \
             GROUP  BY team_fifa_api_id, \
                       team_api_id) src \
         ON t.team_api_id = src.team_api_id \
            AND t.team_fifa_api_id = src.team_fifa_api_id \
order by average desc'
for res in cursor.execute(qry3):print(res)
    
     

# Question 4:  Write a SQL query that ranks all teams by the average of their players' attributes, sort them by descending order displaying only the top 5. 

In [ ]:
print(' Team Name,  Number of Players,  Player Attribute Average ')
print('-'*50)
qry4 = f'SELECT team_long_name, count(player_fifa_api_id), AVG(overall_rating)\
       FROM Team, Player_Attributes WHERE team_fifa_api_id = player_fifa_api_id GROUP BY team_fifa_api_id \
       ORDER BY AVG(overall_rating) DESC LIMIT 5'
for res in cursor.execute(qry4):
    print(res)

# Question 5:  Write a SINGLE SQL query that finds the date that had the most goals scored on, per each different season and league. 

In [ ]:
print(' Date, Season, League Name, Goals scored')
print('-'*70)
qry5 = f"SELECT Match.date, Match.season, League.Name, SUM(Match.home_team_goal + Match.away_team_goal) as Goals \
        FROM Match INNER JOIN LEAGUE ON Match.League_id = League.id \
        GROUP BY Match.Season, League.Name"
for res in cursor.execute(qry5):
    print(res)

# Graduate Student Task: Write a SINGLE SQL query that finds the top 5 teams in terms of goals scored PER league for the 2008/2009 season. 

In [ ]:
print(' Season, League, Rank, Team Name, Goals scored')
print('-'*65)
qry6 = f"SELECT Match.season, League.name, ROW_NUMBER() OVER \
        ( ORDER BY SUM(Match.home_team_goal + Match.away_team_goal) DESC) as \
        Rank,Team.team_long_name, SUM(Match.home_team_goal + Match.away_team_goal) as Goals from match JOIN League \
        ON Match.league_id = League.id \
        JOIN Team ON Team.team_api_id = Match.home_team_api_id OR Team.team_api_id = Match.away_team_api_id \
        WHERE Match.season ='2008/2009' GROUP BY League.name LIMit 5"
for res in cursor.execute(qry6):
    print(res)